# Logic Test Notebook
`run.py`의 실행 로직을 기반으로 특정 프로젝트의 메인 로직을 테스트합니다.

In [1]:
import os
import sys
import importlib
from pkg.utils.blackboard import initialize_global_blackboard, GlobalBlackboard

# 1. 프로젝트 루트 경로 설정 (노트북 위치 기준)
project_root = os.path.abspath('')
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project Root: {project_root}")

# 2. 실행할 프로젝트 이름 설정
PROJECT_NAME = "shimadzu_logic"

# 3. Blackboard 초기화 (run.py와 동일한 경로 규칙 사용)
blackboard_path = os.path.join(project_root, f"projects/{PROJECT_NAME}/configs/blackboard.json")

if os.path.exists(blackboard_path):
    initialize_global_blackboard(blackboard_path)
    print(f"✅ Blackboard initialized from: {blackboard_path}")
else:
    print(f"❌ Blackboard config not found at: {blackboard_path}")

# 4. 프로젝트 메인 모듈 로드 및 실행
module_name = f"projects.{PROJECT_NAME}.main"

try:
    # 소스 코드 수정 시 즉시 반영하기 위해 reload 처리
    if module_name in sys.modules:
        module = importlib.reload(sys.modules[module_name])
    else:
        module = importlib.import_module(module_name)

    if hasattr(module, "main"):
        print(f"🚀 Starting {module_name}.main()...\n")
        module.main()
        bb = GlobalBlackboard()
    else:
        print(f"⚠️ Error: Module '{module_name}' does not define main()")
except Exception as e:
    print(f"❌ Error running project: {e}")
    import traceback
    traceback.print_exc()

Project Root: c:\Users\S\Documents\GitHub\Shimadzu
Blackboard initialized from c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
Blackboard initialized from c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
✅ Blackboard initialized from: c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
🚀 Starting projects.shimadzu_logic.main.main()...

[SYSTEM] Starting shimadzu_logic System...
→ [INFO] Config file loaded successfully: c:\Users\S\Documents\GitHub\Shimadzu\projects\shimadzu_logic\configs\mqtt_config.json
MQTT client initialized: 127.0.0.1:1883

→ [INFO] 브로커 연결 성공!
Tensile Machine MQTT Protocol Connector Started
→ [SUB] 토픽 구독 완료: /ui/cmd
Logic connector is waiting for commands on /ui/cmd.
→ [SUB] 토픽 구독 완료: /binpick/evt


[LogicProcessingThread]: Starting internal logic processing loop.
[StatusPublishingThread]: Starting status publishing loop.
configs robot : 192.168.0.15
configs rem

[device] Manual Test Command Executed: 13
[device] Indicator Down Command Sent Successfully.
[device] Manual Test Command Executed: 12
[device] Indicator Up Command Sent Successfully.
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.178
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.18
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.18
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.18
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.18
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805
[device] Manual Test Command Executed: 9
[device] Dial Gauge Value: 3.1805


In [2]:
bb.get("manual/device/tester")

0

In [35]:
bb.set("manual/device/tester",12)

In [ ]:
# 인디게이터 두께 측정
bb.set("manual/device/tester",9)
bb.get("device/gauge/thickness")

In [3]:
# 두께 측정기 받침 올리기
bb.set("manual/device/tester",12)

In [2]:
# 두께 측정기 받침 내리기
bb.set("manual/device/tester",13)

In [11]:
# 시편 정렬기 풀기
bb.set("manual/device/tester",7)

In [ ]:
# 시편 정렬기 시편 정렬
bb.set("manual/device/tester",6)